In [11]:
import pickle
import pandas as pd
import numpy as np

In [ ]:
with open("assgn_model.bin", "rb") as f_in:
    dv, model = pickle.load(f_in)

In [6]:
categorical = ["PULocationID", "DOLocationID"]

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df["duration"] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df["duration"] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype("int").astype("str")
    
    return df

In [13]:
year = 2023
month = 3

df = read_data(f"https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{year:04d}-{month:02d}.parquet")
df["ride_id"] = f"{year:04d}/{month:02d}_" + df.index.astype("str")

In [9]:
dicts = df[categorical].to_dict(orient="records")
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [22]:
print(np.std(y_pred))

6.247488852238703


In [20]:
df_result = df["ride_id"].to_frame()
df_result["pred"] = y_pred

df_result.to_parquet(
    "output_file.parquet",
    engine="pyarrow",
    compression=None,
    index=False
)